### 📚 Importing Required Libraries

In [105]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 📂 Loading and Consolidating League Season Data

This section reads seasonal league table data from CSV files stored in a folder structure. 
We combine all individual season tables (from 1992–93 up to 2023–24) into a single DataFrame `All_seasons`.

- Each season file is named: `Premier_league_season_table_YYYY-YYYY`
- Each file is read and tagged with its season year using a new column `Season`
- All data is appended into one DataFrame for future analysis


In [106]:
# Initialize parameters
Year = 1992
Current_Year = 2024
No_of_Season = Current_Year -Year

# Initialize counter and empty container
seasons = 0

# Loop through each season and load the data
while seasons <No_of_Season:
    # Creating a season variable for getting the file from directory.
    Season = f'{Year}-{Year+1}'

    # Getting files from directory and reading the files.
    file_path  = f'Datasets/Premier_League/Premier_league_season_table_{Season}'
    Table = pd.read_csv(file_path)
    
    # Creating a Season col in Seasons Points table 
    Table['Season'] = Season
    
    col = Table.columns
    
    if seasons <= 0:
        # Creating a Empty All season dataframe that hold all seasons in one dataframe
        All_seasons = pd.DataFrame(columns=col)

    # Combinding all Points tables into one - ALl_seasons table.
    All_seasons = pd.concat([All_seasons,Table])

    # Move to next season
    Year = Year + 1
    seasons += 1

C:\Users\Admin\AppData\Local\Temp\ipykernel_11656\2154718272.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  All_seasons = pd.concat([All_seasons,Table])


### 🧹 Dropping Unused xG Metrics

The following columns related to expected goals (xG) are not required for the current analysis and are removed to clean the dataset:
- `xG` (Expected Goals)
- `xGA` (Expected Goals Against)
- `xGD` (xG Difference)
- `xGD/90` (xG Difference per 90 minutes)


In [107]:
#  Drop unused xG-related columns
columns_to_drop = ['xG', 'xGA', 'xGD', 'xGD/90']
All_seasons.drop(columns=columns_to_drop, inplace=True)

In [108]:
All_seasons.head()

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,Attendance,Top Team Scorer,Goalkeeper,Notes,Season
0,1,Manchester Utd,42,24,12,6,67,31,36,84,2.00,"35,132",Mark Hughes - 15,Peter Schmeichel,→ Champions League via league finish,1992-1993
1,2,Aston Villa,42,21,11,10,57,40,17,74,1.76,"29,594",Dean Saunders - 12,Nigel Spink,→ UEFA Cup via league finish,1992-1993
2,3,Norwich City,42,21,9,12,61,65,-4,72,1.71,"16,253",Mark Robins - 15,Bryan Gunn,→ UEFA Cup via league finish 1,1992-1993
3,4,Blackburn,42,20,11,11,68,46,22,71,1.69,"16,247",Alan Shearer - 16,Bobby Mimms,NaN,1992-1993
4,5,QPR,42,17,12,13,63,55,8,63,1.50,"15,001",Les Ferdinand - 20,Tony Roberts,NaN,1992-1993


In [109]:
All_seasons.info()

<class 'pandas.core.frame.DataFrame'>
Index: 646 entries, 0 to 19
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Rk               646 non-null    object 
 1   Squad            646 non-null    object 
 2   MP               646 non-null    object 
 3   W                646 non-null    object 
 4   D                646 non-null    object 
 5   L                646 non-null    object 
 6   GF               646 non-null    object 
 7   GA               646 non-null    object 
 8   GD               646 non-null    object 
 9   Pts              646 non-null    object 
 10  Pts/MP           646 non-null    float64
 11  Attendance       646 non-null    object 
 12  Top Team Scorer  646 non-null    object 
 13  Goalkeeper       646 non-null    object 
 14  Notes            326 non-null    object 
 15  Season           646 non-null    object 
dtypes: float64(1), object(15)
memory usage: 85.8+ KB


In [110]:
All_seasons['Squad'].unique()

array([' Manchester Utd', ' Aston Villa', ' Norwich City', ' Blackburn',
       ' QPR', ' Liverpool', ' Sheffield Weds', ' Tottenham',
       ' Manchester City', ' Arsenal', ' Chelsea', ' Wimbledon',
       ' Everton', ' Sheffield Utd', ' Coventry City', ' Ipswich Town',
       ' Leeds United', ' Southampton', ' Oldham Athletic',
       ' Crystal Palace', ' Middlesbrough', " Nott'ham Forest",
       ' Newcastle Utd', ' West Ham', ' Swindon Town', ' Leicester City',
       ' Bolton', ' Derby County', ' Sunderland', ' Barnsley',
       ' Charlton Ath', ' Bradford City', ' Watford', ' Fulham',
       ' Birmingham City', ' West Brom', ' Portsmouth', ' Wolves',
       ' Wigan Athletic', ' Reading', ' Stoke City', ' Hull City',
       ' Burnley', ' Blackpool', ' Swansea City', ' Cardiff City',
       ' Bournemouth', ' Brighton', ' Huddersfield', ' Brentford',
       'Manchester City', 'Arsenal', 'Manchester Utd', 'Newcastle Utd',
       'Liverpool', 'Brighton', 'Aston Villa', 'Tottenham', 'B

### Key observations.
- Squad columns contain teams names have an unwanted space which is recognising the same team as different teams.  
- Top Team Scorer Contains both name and goal count (e.g. "Mark Hughes - 15") — could be split for clarity.
- Notes column contains NaNs
- Values in Attendance column looks Numeric but has commas — may be stored as strings.

For making changes into these columns we need to do some kind of operations 

#### Removing the Space from teams

In [111]:
All_seasons['Squad'] = All_seasons['Squad'].str.strip()

#### Split 'Top Team Scorer' into name and goals

In [112]:
def top_scorer_and_goals(season):
    # Splitting Goal scorers and number of goals scored by Top scorer   
    
    season["Team Top_scorer_Goals"] = season['Top Team Scorer'].apply(lambda x : int(x.split(" - ")[-1]))
    
    season['Team_Top_Scorer'] = season['Top Team Scorer'].apply(lambda x : x.split(" - ")[0])
    All_seasons.drop(columns=['Top Team Scorer'],inplace=True)
                                                                
    return season

In [113]:
All_seasons = top_scorer_and_goals(All_seasons)

#### Convert Attendance to Numeric

In [114]:
def Attendance_dtype(season):
    season['Attendance'].apply(lambda x : int(x.split(",")[0] + x.split(",")[-1]) if type(x) == str else x)

In [115]:
Attendance_dtype(All_seasons)

### Initializing Columns for Promotion & Performance Tracking

To track team performance trends, we introduce the following columns:
- `Promoted`: Indicates if a team was promoted in that season.
- `Years_in_League_after_promotion`: Tracks consecutive seasons in top flight after promotion.
- `Same_Rank_As_Last_Season`: Flags if a team retained the same position as last y

In [116]:
# Initialize tracking columns for promotion and consistency analysis
All_seasons['Promoted'] = np.nan  # Will be filled with 'Yes' or 'No'
All_seasons['Years_in_League_after_promotion'] = 1  # Starting value; will be updated per team
All_seasons['Same_Rank_As_Last_Season'] = 0  # Will be 1 if rank matches previous season

#### Consecutive_Years_In_League_After_Promotion

This function helps track the number of consecutive years a team has stayed in the Premier League after being promoted.

In [117]:
def Consecutive_Years_In_League_After_Promotion(current_season_df, last_season_df):
    """
    Adds the 'Years_in_League_after_promotion' column to the current season DataFrame,
    tracking how many consecutive years a team has been in the league.

    Parameters:
    - current_season_df (pd.DataFrame): DataFrame of the current season.
    - last_season_df (pd.DataFrame): DataFrame of the previous season.

    Returns:
    - pd.DataFrame: Modified current season DataFrame with the new column.
    """
    # Initialize the column with default value

    # Create dictionary from last season to map each team to its previous year count
    last_years_dict = last_season_df.set_index('Squad')['Years_in_League_after_promotion'].to_dict()

    # For each row, update the value based on last season
    for idx, row in current_season_df.iterrows():
        team = row['Squad']
        if team in last_years_dict:
            current_season_df.at[idx, 'Years_in_League_after_promotion'] = last_years_dict[team] + 1
        else:
            current_season_df.at[idx, 'Years_in_League_after_promotion'] = 1  # New team or promoted

    return current_season_df


#### Promotion

This function flags whether a team in the current season was promoted from a lower division.

In [118]:
def promotion(current_season, last_season):
    """
    Adds a 'Promoted' column to the current_season DataFrame indicating whether 
    each team was promoted (not present in last season's data).
    
    Parameters:
    - current_season (pd.DataFrame): DataFrame of the current season.
    - last_season (pd.DataFrame): DataFrame of the previous season.
    
    Returns:
    - pd.DataFrame: Modified current season DataFrame with 'Promoted' column.
    """
    last_season_teams = set(last_season['Squad'])
    current_season['Promoted'] = current_season['Squad'].apply(
        lambda team: "Yes" if team not in last_season_teams else "No"
    )
    return current_season


#### Mark same rank as last season

This function checks whether a team held the same league position (rank) in two consecutive seasons.

In [119]:
def mark_same_rank_as_last_season(previous_season, current_season):
    """
    Compares team rankings between two seasons.
    
    Args:
        previous_season (DataFrame): Last season's league table with 'Squad' and 'Rk' columns.
        current_season (DataFrame): Current season's league table with 'Squad' and 'Rk' columns.
    
    Returns:
        DataFrame: A copy of the current season with a new column:
                  'Same_Rank_As_Last_Season' (1 if same rank, else 0).
    """
     # Step 1: Create a mapping of previous season's team ranks   
    previous_season_ranks = previous_season.set_index('Squad')['Rk'].to_dict()
    
    # Step 2: Apply the comparison using a lambda function
    current_season = current_season.copy()
    current_season['Same_Rank_As_Last_Season'] = current_season.apply(
        lambda row: 1 if previous_season_ranks.get(row['Squad']) == row['Rk'] else 0,
        axis=1
    )
    return current_season


#### Looping Through All Seasons: Enriching the Data

In this section, we iterate through every Premier League season from 1992–93 to the current year and perform several enhancements on each season's data. The goal is to derive meaningful historical context for each team.

🪜 Step-by-step Breakdown:
1. Sorting the Data:
We begin by sorting All_seasons by Season and Rk (Rank) to maintain chronological and table-order consistency.
2. Creating a List of Unique Seasons:
We extract all unique season identifiers to loop through them sequentially.

3. Processing Each Season Individually with above all 3 functions:
 
4. Combining All Processed Seasons:
After processing all seasons, we merge them back into a single DataFrame All_seasons containing new contextual columns.


In [120]:
# Ensure seasons are sorted properly
All_seasons = All_seasons.sort_values(by=['Season','Rk']).reset_index(drop=True)

# List to store modified DataFrames
season_list = []

# Get unique season years
unique_seasons = All_seasons['Season'].unique()

for i in range(len(unique_seasons)):
    
    current_season_year = unique_seasons[i]
    current_season_df = All_seasons[All_seasons['Season'] == current_season_year].copy()
    
    if i == 0:
        # First season: initialize columns
        current_season_df['Promoted'] = "No"
        current_season_df['Years_in_League_after_promotion'] = 1
        current_season_df['Same_Rank_As_Last_Season'] = 0
    else:
        last_season_year = unique_seasons[i - 1]
        last_season_df = season_list[i - 1].copy()  # Use already processed data
        
        # Ensure the required column exists
        if 'Years_in_League_after_promotion' not in last_season_df.columns:
            last_season_df['Years_in_League_after_promotion'] = 1
        
        # Apply all three functions
        current_season_df = promotion(current_season_df, last_season_df)
        current_season_df = Consecutive_Years_In_League_After_Promotion(current_season_df, last_season_df)
        current_season_df = mark_same_rank_as_last_season(last_season_df, current_season_df)
    
    # Append to the result list
    season_list.append(current_season_df)

# Final combined DataFrame
All_seasons = pd.concat(season_list).reset_index(drop=True)


In [122]:
All_seasons.tail(40)

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,Attendance,Goalkeeper,Notes,Season,Team Top_scorer_Goals,Team_Top_Scorer,Promoted,Years_in_League_after_promotion,Same_Rank_As_Last_Season
606,1,Manchester City,38,28,5,5,94,33,61,89,2.34,53249,Ederson,→ Champions League via league finish,2022-2023,36,Erling Haaland,No,21,1
607,2,Arsenal,38,26,6,6,88,43,45,84,2.21,60191,Aaron Ramsdale,→ Champions League via league finish,2022-2023,15,"Martin Ødegaard, Gabriel Martinelli",No,31,0
608,3,Manchester Utd,38,23,6,9,58,43,15,75,1.97,73671,David de Gea,→ Champions League via league finish,2022-2023,17,Marcus Rashford,No,31,0
609,4,Newcastle Utd,38,19,14,5,68,33,35,71,1.87,52127,Nick Pope,→ Champions League via league finish,2022-2023,18,Callum Wilson,No,6,0
610,5,Liverpool,38,19,10,9,75,47,28,67,1.76,53163,Alisson,→ Europa League via league finish,2022-2023,19,Mohamed Salah,No,31,0
611,6,Brighton,38,18,8,12,72,53,19,62,1.63,31477,Robert Sánchez,→ Europa League via league finish,2022-2023,10,Alexis Mac Allister,No,6,0
612,7,Aston Villa,38,18,7,13,51,46,5,61,1.61,41679,Emiliano Martínez,→ Europa Conference League via league finish,2022-2023,15,Ollie Watkins,No,4,0
613,8,Tottenham,38,18,6,14,70,63,7,60,1.58,61585,Hugo Lloris,NaN,2022-2023,30,Harry Kane,No,31,0
614,9,Brentford,38,15,14,9,58,46,12,59,1.55,17078,David Raya,NaN,2022-2023,20,Ivan Toney,No,2,0
615,10,Fulham,38,15,7,16,55,53,2,52,1.37,23746,Bernd Leno,NaN,2022-2023,14,Aleksandar Mitrović,Yes,1,0


#### Saving the Final Dataset

After enriching and combining all seasonal league tables, we save the final All_seasons DataFrame to a CSV file for future use or analysis.

In [123]:
All_seasons.to_csv(f'Datasets/Premier_League/All_seasons',index=False)